In [ ]:
# Outside imports
import os
import importlib
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
# load real data
data_path = '/Users/rowancassius/Desktop/capstone/data/context_task_data.tsv'
dat = pd.read_csv(data_path, sep='\t')

In [ ]:
import preprocess 
importlib.reload(preprocess)
from preprocess import *

In [ ]:
!pwd

In [ ]:
# dat

Summary Modifications:
* lowercase first letter
* lowercase it and this
* filter punctuation

Article Modifications (ideal):
* lowercase non-entities
* 

In [ ]:
from nltk.tokenize import word_tokenize, wordpunct_tokenize

In [ ]:
s = "Remind him that my name is Ro-Ro, and this is Catherine's channel by the way, and you can reach me at 415-279-3799 or you can contact me at rowancassius@gmail.com or at my personal website which is www.yougotitMAN.com. By the way I may also work for Google at Mountain View CA. What do you think?"

In [ ]:
art = 'Google jus tiisued us 5 times that amount captain. Please do bbetter'

In [ ]:
ENT_TAGS = {
    'PERSON',
    'NORP',
    'FAC',
    'ORG',
    'GPE',
    'LOC',
    'PRODUCT',
    'EVENT'
    'WORK_OF_ART',
    'LAW',
    'LANGUAGE',
    'DATE',
    'TIME',
    'PERCENT',
    'MONEY',
    'QUANTITY',
    'ORDINAL',
    'CARDINAL'
}

CAP_WORDS = {'SENDER', 'I'}.union(INFO_CATS, ENT_TAGS)

In [ ]:
import en_core_web_sm
nlp = en_core_web_sm.load()


def process_text(text):
    text = clean_info(text)
    text = replace_NE(text)
    text = re.sub(BLACK, '', text)
    tokens = [t if t in CAP_WORDS else t.lower() for 
              t in word_tokenize(text)]
    return tokens


def replace_NE(text):
    'Replace the named entities with their types'
    ent_lab = {e.text: e.label_ for e in nlp(text).ents if 
               e.text not in INFO_CATS}
    for ent, lab in ent_lab.items():
        text = re.sub(ent, lab, text)
    return text

In [ ]:
CAP_TAGS = {'SENDER', 'I'}.union(ENT_TAGS)

In [ ]:
CAP_TAGS = ENT_TAGS + 

In [ ]:
tokens = [t if t in CAP_TAGS else t.lower() for t in word_tokenize(sent)]
tokens

In [ ]:
s = "The guy I'm watching on Home Watchers right 134 is Incredibly annoying SENDER who is at 6488-687. Yes, Please repeat that Arnold at 3:53."

In [ ]:
process_text(s)

In [ ]:
# NON_ENT = '|'.join(ENT_TAGS)
# ps = process_sent(s)

re.findall('(?!Incredibly)\w+', s)

In [ ]:
ps

In [ ]:
re.findall('|'.join(ENT_TAGS), lambdaprocess_sent(s))